In [1]:
import asyncio

# Introduction

A primer on how to use asynchronous programming in python via the inbuilt `asyncio` package.

Note that asynchrnous programming __is not__ the same as multithreading. The latter refers to creating multiple threads to _run several processes concurrently_. 

Asynchronous programming only performs __one process at a time__, however when a certain task is paused (awaiting a response / sleeping / etc), the processor can be given to another task so as to minimize the overall compute time required. 

##  A note on running `async` in jupyter notebooks

Jupyter notebooks have a persistent `asyncio` [loop running](https://ipython.readthedocs.io/en/stable/interactive/autoawait.html#difference-between-terminal-ipython-and-ipykernel). This means that you will not be able to execute asyncronous code in notebook cells using `asyncio.run()`. You may simply call `await function()` instead. 

Think of it as there being a `asyncio.run()` command inherent to each executed cell. 

Note that the examples listed here will be with respect to asynchronous programming in genenral python environments, not in jupyter.

# Co-Routines

A co-routine is a wrapper for an asynchronous python function. It turns a traditional python function into a program component that can be suspended or resumed on command. 

The declaration of a coroutine is simple. Add the `async` statement before any function definition. 

In [3]:
# This is a coroutine
async def main():
    print("Hell world!")
   


# Event Loops

Calling a coroutine in the traditional python style, for example :
```{python}
async def main():
    print("Hell world!")

if __name__ == "__main__"
    main()
```
will not work. 

Executing a coroutine can only be done in an event loop. In computer science, the event loop is a construct that waits for and dispatches events or messages in a program. 

The `async` package has an inbuilt event loop constructor (`async.run()`) that handles the lower level implementation details for us. 

The example above is thus rectified to :

```{python}
async def main():
    print("Hell world!")

if __name__ == "__main__"
    asyncio.run(main())
```

All coroutines (asynchrnous python functions) can also be called within other coroutines, however they must be called via the `await` syntax. More on that in the next section.

# `await`

The `await` syntax is a key part of the asynchronous framework in python. It allows coroutines to be called in other coroutines. 

The only way to call another coroutine within a coroutine is by invoking it with `await`. This will force the program to wait until the called coroutine is complete before moving on.

In [8]:
async def main():
    print("hello there!")
    await foo()
    print("All done!")

async def foo():
    print("text")
    await asyncio.sleep(1)

await main()

hello there!
text
All done!


# Tasks

From the above example we can see that `await` forces the program to wait till the coroutine it calls has been completed. 

Ideally if a coroutine is not doing anything (e.g. sleeping / awaiting a http response), we want the processor to be performing other operations. 

We can accomplish this by using `tasks` in `asyncio`. This will assign a task in the program to execute the specified coroutine __as soon as possible__. 

In [14]:
async def main():
    print("hello there!")
    task = asyncio.create_task(foo())
    print("All done!")

async def foo():
    print("text")
    await asyncio.sleep(1)

await main()

hello there!
All done!
text


`"All done!"` gets printed before `'text'` because even though the task has been created, the `main()` function is still running. Remember that created `tasks` are only executed once the processor becomes available. 

In the case of the example this was at the end of the `main()` function call.

In [15]:
async def main():
    print("hello there!")
    task = asyncio.create_task(foo())
    await asyncio.sleep(2)
    print("All done!")

async def foo():
    print("text")
    await asyncio.sleep(1)

await main()

hello there!
text
All done!


In this new example we can see that since the `main()` function had an idle period of 2 seconds between creating the task and printing `"All done!"`, it reassigned the processor to the completion of `foo()`.

# Futures

Most coroutines are run with a return value in mind. When a coroutine is run as a `task`, a `future` is created. This is a placeholder value for what the coroutine will actually return in the future once it has been completed. 

In [19]:
async def fetch_data():
    print("Start fetching")
    await asyncio.sleep(2)
    return {'data':1}

async def print_numbers():
    for i in range(5):
        print(i)
        await asyncio.sleep(0.25)

async def main():
    task1 = asyncio.create_task(fetch_data())
    # task2 = asyncio.create_task(print_numbers())
    print(task1)

await main()


<Task pending name='Task-27' coro=<fetch_data() running at /var/folders/lt/1bfps6v903d64dxtxrz736pw0000gn/T/ipykernel_94129/1358838374.py:1>>
Start fetching


Observe that this does not actually print out the proper return value of `fetch_data()`

To get the proper return value we will need to force the completion of the task using `await`

In [20]:
async def main():
    task1 = asyncio.create_task(fetch_data())
    # task2 = asyncio.create_task(print_numbers())
    value = await task1
    print(value)

await main()

Start fetching
{'data': 1}


Note that all scheduled tasks will always complete by the time the program terminates. That is to say that the program will not terminate while there are still unfinished tasks.

Not sure if this behaviour differs from previous python versions. Some of the resources make it appear that tasks may be left unfinished at termination time if not explicitly awaited for. However my trials both on jupyter & the command line python interpreter always complete all tasks before termination.

# Resources

1. Excellent introductory [youtube video](https://www.youtube.com/watch?v=t5Bo1Je9EmE)

2. [Docs](https://docs.python.org/3/library/asyncio.html) are not super beginer-friendly, will require some details reading.